In [12]:
# nltk module
import nltk
from nltk.stem.lancaster import LancasterStemmer

# module we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random
import json


Using TensorFlow backend.


In [2]:
# create a stemmer
stemmer = LancasterStemmer()

# load intents
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)




In [3]:
words = []
classes = []
documents = []
ignore_words = []


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word into a sentence
        w = nltk.word_tokenize(pattern)
        # add word to our words list
        words.extend(w)
        # add documents to our corpus
        documents.append((w, intent['tag']))

        # add to our classses
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# stem and lower case the word and remove duplicates
words =  [stemmer.stem(w.lower()) for w in words ]




# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))

In [4]:

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

45 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
83 unique stemmed words ["'s", ',', '?', 'a', 'adv', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'cau', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


In [5]:
# # create our training data
# training = []
# # create an empty array for our output
# output_empty = [0] * len(classes)
# for doc in documents:
#     bag = []
#     pattern_words = doc[0]
#     pattern_words = [stemmer.stem(word.lower()) for word in pattern_words ]
    
#     for w in words:
#         bag.append(1) if w in pattern_words else bag.append(0)
        
#     output_row = list(output_empty)
#     output_row[classes.index(doc[1])] = 1
#     traininga.append(bag, output_row)


# output_row

In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

In [7]:
training[0]

array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
       list([0, 0, 0, 1, 0, 0, 0, 0, 0])], dtype=object)

In [10]:

# create train and test lists. X - patterns, Y - intents

train_x = list(training[:,0])

train_y = list(training[:,1])

print(train_x[:1], train_y[:1])
len(train_x), len(train_y)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]] [[0, 0, 0, 1, 0, 0, 0, 0, 0]]


(45, 45)

In [13]:

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons

# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [14]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               10752     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 585       
Total params: 19,593
Trainable params: 19,593
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Fit the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
45/45 [==============================] - 0s 4ms/step - loss: 2.2034 - accuracy: 0.2000
Epoch 2/200
45/45 [==============================] - 0s 257us/step - loss: 2.0865 - accuracy: 0.2222
Epoch 3/200
45/45 [==============================] - 0s 263us/step - loss: 2.0606 - accuracy: 0.2222
Epoch 4/200
45/45 [==============================] - 0s 203us/step - loss: 1.9922 - accuracy: 0.2889
Epoch 5/200
45/45 [==============================] - 0s 220us/step - loss: 1.9119 - accuracy: 0.4444
Epoch 6/200
45/45 [==============================] - 0s 282us/step - loss: 1.7127 - accuracy: 0.4667
Epoch 7/200
45/45 [==============================] - 0s 257us/step - loss: 1.6455 - accuracy: 0.5333
Epoch 8/200
45/45 [==============================] - 0s 194us/step - loss: 1.4391 - accuracy: 0.6000
Epoch 9/200
45/45 [==============================] - 0s 225us/step - loss: 1.4207 - accuracy: 0.5778
Epoch 10/200
45/45 [==============================] - 0s 285us/step - loss: 1.2097 - accuracy

45/45 [==============================] - 0s 226us/step - loss: 0.0558 - accuracy: 1.0000
Epoch 82/200
45/45 [==============================] - 0s 250us/step - loss: 0.0634 - accuracy: 1.0000
Epoch 83/200
45/45 [==============================] - 0s 220us/step - loss: 0.0552 - accuracy: 0.9778
Epoch 84/200
45/45 [==============================] - 0s 211us/step - loss: 0.0898 - accuracy: 0.9778
Epoch 85/200
45/45 [==============================] - 0s 249us/step - loss: 0.0522 - accuracy: 1.0000
Epoch 86/200
45/45 [==============================] - 0s 234us/step - loss: 0.0411 - accuracy: 1.0000
Epoch 87/200
45/45 [==============================] - 0s 151us/step - loss: 0.0208 - accuracy: 1.0000
Epoch 88/200
45/45 [==============================] - 0s 249us/step - loss: 0.0438 - accuracy: 0.9778
Epoch 89/200
45/45 [==============================] - 0s 219us/step - loss: 0.0873 - accuracy: 0.9778
Epoch 90/200
45/45 [==============================] - 0s 177us/step - loss: 0.0192 - accuracy: 

45/45 [==============================] - 0s 191us/step - loss: 0.0222 - accuracy: 1.0000
Epoch 161/200
45/45 [==============================] - 0s 505us/step - loss: 0.0073 - accuracy: 1.0000
Epoch 162/200
45/45 [==============================] - 0s 279us/step - loss: 0.0144 - accuracy: 1.0000
Epoch 163/200
45/45 [==============================] - 0s 145us/step - loss: 0.0114 - accuracy: 1.0000
Epoch 164/200
45/45 [==============================] - 0s 237us/step - loss: 0.0894 - accuracy: 0.9778
Epoch 165/200
45/45 [==============================] - 0s 155us/step - loss: 0.0272 - accuracy: 0.9778
Epoch 166/200
45/45 [==============================] - 0s 264us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 167/200
45/45 [==============================] - 0s 310us/step - loss: 0.0194 - accuracy: 1.0000
Epoch 168/200
45/45 [==============================] - 0s 238us/step - loss: 0.0185 - accuracy: 1.0000
Epoch 169/200
45/45 [==============================] - 0s 146us/step - loss: 0.0475 - a

In [16]:
# save model to file
pickle.dump(model, open("./pickle/ahsbot-model.pkl", "wb"))

# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "./pickle/ahsbot-data.pkl", "wb" ) )